# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record



In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import seaborn as sns

%matplotlib inline

# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Assesment

In [3]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [5]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


- max value of age is  unusual

In [7]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


- null values in gender and income
- datatype for become_member_on

In [9]:
profile[(profile.gender.notnull())&(profile.income.notnull())].shape[0]

14825

- null values in income is associated with null values in gender

In [10]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


- unsuitable encoding for value feature

In [11]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


## Cleaning

1. Drop null values from profile dataframe
2. convert datatype of became member on to datetime

In [12]:
profile_clean=profile.dropna(axis=0)
profile_clean.became_member_on=pd.to_datetime(profile_clean.became_member_on,format='%Y%m%d')

C:\Users\gr1\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


**test**

In [13]:
profile_clean.info()
profile_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   gender            14825 non-null  object        
 1   age               14825 non-null  int64         
 2   id                14825 non-null  object        
 3   became_member_on  14825 non-null  datetime64[ns]
 4   income            14825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 694.9+ KB


,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0


In [17]:
profile_clean.describe()

,age,income
count,14825.000000,14825.000000
mean,54.393524,65404.991568
std,17.383705,21598.299410
min,18.000000,30000.000000
25%,42.000000,49000.000000
50%,55.000000,64000.000000
75%,66.000000,80000.000000
max,101.000000,120000.000000


In [19]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


<br>
3. encoding value feature in transcript to seperated columns

In [20]:
transcript_clean=pd.concat([transcript,pd.json_normalize(transcript.value)],axis=1).drop(columns='value')
transcript_clean.head()

,person,event,time,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [21]:
transcript_clean=transcript_clean.fillna('')
transcript_clean.offer_id=transcript_clean['offer id']+transcript_clean['offer_id']
transcript_clean.drop(columns=['offer id','reward'],inplace=True)
transcript_clean.replace({'':np.nan},inplace=True)

In [22]:
transcript_clean.head()

,person,event,time,amount,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


In [23]:
event_classes = transcript_clean.event.unique()
eclasses = pd.api.types.CategoricalDtype(ordered = True, categories = event_classes)
transcript_clean['event'] = transcript_clean['event'].astype(eclasses)

In [24]:
transcript_clean=pd.melt(transcript_clean, id_vars=['person','event','time'],
        var_name='type', value_name='value').dropna(axis=0).sort_values(['time','person','event']).reset_index(drop=True)

In [25]:
#add offer's type to dataframe and it will replace type column
transcript_clean=transcript_clean.join(portfolio[['id','offer_type']].set_index('id'),on='value').replace({'offer_id':'',np.nan:''})

transcript_clean['type']=transcript_clean['type']+transcript_clean['offer_type']
transcript_clean.drop(columns='offer_type',inplace=True)
transcript_clean.head()

,person,event,time,type,value
0,0011e0d4e6b944f998e987f904e8c1e5,offer received,0,informational,3f207df678b143eea3cee63160fa8bed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,discount,fafdcd668e3743c1bb461111dcafc2a4
2,003d66b6608740288d6cc97a6903f4f0,offer received,0,informational,5a8bc65990b245e5a138643cd4eb9837
3,00426fe3ffde4c6b9cb9ad6d077a13ea,offer received,0,informational,5a8bc65990b245e5a138643cd4eb9837
4,005500a7188546ff8a767329a2f7c76a,offer received,0,bogo,ae264e3637204a6fb9bb56bc8210ddfd


In [19]:
transcript_clean.type.unique()

array(['informational', 'discount', 'bogo', 'amount'], dtype=object)

In [20]:
transcript_clean=transcript_clean.join(portfolio[['reward','difficulty','duration','id']].set_index('id'),on='value')

transcript_clean.loc[transcript_clean.event!='offer received',['reward','difficulty','duration']]=0

transcript_clean.replace({"reward":{0:np.nan},"difficulty":{0:np.nan},"duration":{0:np.nan}},inplace=True)

transcript_clean.head()

,person,event,time,type,value,reward,difficulty,duration
0,0011e0d4e6b944f998e987f904e8c1e5,offer received,0,informational,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,discount,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0
2,003d66b6608740288d6cc97a6903f4f0,offer received,0,informational,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,3.0
3,00426fe3ffde4c6b9cb9ad6d077a13ea,offer received,0,informational,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,3.0
4,005500a7188546ff8a767329a2f7c76a,offer received,0,bogo,ae264e3637204a6fb9bb56bc8210ddfd,10.0,10.0,7.0


In [21]:
def offer_features(previous_transactions):
    '''caluclate number of every transaction[received,view,completed]
    related to each offer type and fist and last time of this event'''
    
    df=pd.DataFrame(columns=['first_bogo_received','first_discount_received','first_informational_received','first_bogo_viewed',
                     'first_discount_viewed','first_informational_viewed','first_bogo_completed','first_discount_completed',
                     'last_bogo_received','last_discount_received','last_informational_received','last_bogo_viewed',
                     'last_discount_viewed','last_informational_viewed','last_bogo_completed','last_discount_completed',
                     'count_bogo_received','count_discount_received','count_informational_received','count_bogo_viewed',
                     'count_discount_viewed','count_informational_viewed','count_bogo_completed','count_discount_completed'],
                   data=[[-1 for _ in range(16)]+[0 for _ in range(8)]])
    
    temp=previous_transactions[previous_transactions['type']!='amount']
    if temp.shape[0]==0:
        return df
    temp=temp.groupby(['person','type','event'],observed=True,sort=False).agg({'time':['first','last'],
                                               'event':'count'}).unstack(level=-1,fill_value=0).unstack(level=-1,fill_value=0)

    #convert multi dimension columns to one dimension and rename them
    temp.columns=temp.columns.map('{0[1]}_{0[3]}_{0[2]}'.format).str.replace('offer ','')

    #drop all columns related to complete informational event as it will always be null
    drop=temp.columns.str.contains('informational_completed')
    temp=temp.drop(columns=temp.columns[drop])

    return df.append(temp.reset_index(drop=True)).fillna(method='ffill').iloc[-1:,:]

In [22]:
def duration_features(previous_transactions):
    '''calculate features related to duration and reward and difficulty for every offer '''
    df= pd.DataFrame(columns=['max_duration_bogo','max_duration_discount','max_duration_informational','min_duration_bogo',
 'min_duration_discount','min_duration_informational','mean_duration_bogo','mean_duration_discount','mean_duration_informational',
 'median_duration_bogo','median_duration_discount','median_duration_informational','sum_duration_bogo','sum_duration_discount',
 'sum_duration_informational','max_reward_bogo','max_reward_discount','min_reward_bogo','min_reward_discount','mean_reward_bogo',
 'mean_reward_discount','median_reward_bogo','median_reward_discount','sum_reward_bogo','sum_reward_discount','max_difficulty_bogo',
 'max_difficulty_discount','min_difficulty_bogo','min_difficulty_discount','mean_difficulty_bogo','mean_difficulty_discount',
 'median_difficulty_bogo','median_difficulty_discount','sum_difficulty_bogo','sum_difficulty_discount']
                   , data=[[0 for _ in range(35)]])
    #duration feature for evey offer
    offer_df=previous_transactions[previous_transactions['type']!='amount']
    if offer_df.shape[0]== 0:
        return df
    temp=offer_df.groupby(['person','type'],sort=False,observed=True).agg({'duration':['max','min',
                                                                                   'mean','median','sum']}).unstack(level=-1,fill_value=0)
    temp.columns=temp.columns.map('{0[1]}_{0[0]}_{0[2]}'.format) #convert multi dimension columns to one dimension
    row_df = temp.reset_index(drop=True)
    
    #reward and difficulty for every offer
    temp=offer_df.groupby(['person','type'],sort=False).agg({'reward':['max','min','mean','median','sum'],
                                  'difficulty':['max','min','mean','median','sum']}).unstack(level=-1,fill_value=0)
    temp.columns=temp.columns.map('{0[1]}_{0[0]}_{0[2]}'.format)
    
    #drop any column related to informational offer as it doesn't have reward or difficulty
    drop=temp.columns.str.contains('informational') 
    temp=temp.drop(columns=temp.columns[drop])

    return df.append(pd.concat([row_df,temp.reset_index(drop=True)],axis=1)).fillna(method='ffill').iloc[-1:,:]

In [23]:
def is_offer_completed(next_transactios):
    
    #get the cuurent record where we want to know if offer completed or not
    current_transaction=next_transactios.iloc[0,:]
    if current_transaction['type']== 'informational':
        current_values=current_transaction[['time','duration','type']].values
        temp=next_transactios.loc[(next_transactios.time >= current_values[0]) 
                                &(next_transactios.time <= current_values[0]+current_values[1]*24)
                                &(next_transactios['type'].isin(['amount','informational']) )] ['event'].values
        return all(item in temp for item in ['offer viewed','transaction'])
    else:
        current_values=current_transaction[['time','duration','value']].values
        temp=next_transactios.loc[(next_transactios.time >= current_values[0])
                  &(next_transactios.time<= current_values[0]+current_values[1]*24)
                  &(next_transactios.value==current_values[2])]['event'].values

        return all(item in temp for item in ['offer viewed','offer completed'])

In [24]:
def amount_features(previous_transactions):
    '''calculate all features related to money
    max ,min,mean,median,sum,count,last amount customer has paid
    and first and last time'''
    df_tra=pd.DataFrame(columns=['time_first_transaction','time_last_transaction','count_transaction','min_transaction',
                     'max_transaction','mean_transaction','median_transaction','sum_transaction','last_transaction']
                   ,data=[[-1,-1,0,0,0,0,0,0,0]])
    df=previous_transactions[previous_transactions['type']=='amount']
    if df.shape[0]==0:
        return df_tra
    df=df.astype({'value':float})
    
    temp=df.groupby(['person','event'],observed=True,sort=False).agg({'time':['first','last'],
                                               'value':['count','min','max','mean','median','sum','last']}).unstack(level=-1,fill_value=0)

    temp.columns=temp.columns.map('{0[0]}_{0[1]}_{0[2]}'.format).str.replace('value_','')
    return temp.reset_index(drop=True)

In [25]:
def get_training_data(transcript_clean):
    grouped_persons=transcript_clean.groupby('person',sort=False)
    persons_id=transcript_clean.person.unique()
    data=pd.DataFrame()
    
    for i,p in enumerate(persons_id):
        print(i)
        person_transactions=grouped_persons.get_group(p)
        received_offers=person_transactions[person_transactions.event == 'offer received'].copy()
        for i in received_offers.index:
            previous_transactions = person_transactions.loc[:i-1,:]
            features_1 = offer_features(previous_transactions)
            features_2 = duration_features(previous_transactions)

            features_3 = amount_features(previous_transactions)

            received_offers.loc[i:i,'success'] = is_offer_completed(person_transactions.loc[i:,:])
            if data.shape[0]==0:
                data=pd.concat([received_offers.loc[i:i , :].reset_index(),features_1,
                                features_2,features_3],axis=1)
            else:
                data=data.append(pd.concat([received_offers.loc[i:i , :].reset_index(),
                                            features_1,features_2,features_3],axis=1))
            
            
    return data.reset_index(drop=True)   
        
    
    

In [83]:
train_test_df=get_training_data(transcript_clean)

In [944]:
#replace value of success column
train_test_df.replace({False:0,True:1},inplace=True)

In [994]:
#estimate number of completed informational completed for every record
train_test_df.loc[:,'count_informational_completed']=0
train_test_df.loc[(train_test_df.type=='informational') & (train_test_df.success==1),'count_informational_completed']=1
temp1=train_test_df.groupby('person',sort=False,as_index=False).count_informational_completed.cumsum().values
temp2=train_test_df.loc[:,'count_informational_completed'].values.reshape(-1,1)
train_test_df['count_informational_completed']=temp1-temp2

In [1121]:
#join with inforamtion of every customer
train_test_df=train_test_df.join(profile_clean.set_index('id'),on='person')
#drop null values in gender and income
train_test_df.replace({"reward":{np.nan:0},"difficulty":{np.nan:0},"duration":{np.nan:0}},inplace=True)
train_test_df.dropna(axis=0,inplace=True)

In [1123]:
train_test_df.shape

(66501, 83)

In [122]:
#calculate when first_informational_completed and last_informational_completed
temp=train_test_df.copy()

temp=temp.sort_values(['person','index'])

temp=temp.loc[(temp.type=='informational')&(temp.success==1)]

first=temp.groupby('person',sort=False,as_index=False).first()

first.loc[:,'first_informational_completed']=first.loc[:,'time'].values.reshape(-1,1)

train_test_df=train_test_df.join(first.set_index('index')['first_informational_completed'],on='index')

train_test_df.first_informational_completed=train_test_df.groupby('person',
                                            sort=False).first_informational_completed.shift(periods=1)



train_test_df.first_informational_completed=train_test_df.groupby('person',
                                                             sort=False).first_informational_completed.fillna(method='ffill')

In [125]:
temp=train_test_df.copy()

temp=temp.sort_values(['person','index'])
temp=temp.loc[(temp.type=='informational')]
temp.loc[(temp.success==1),'last_informational_completed']=temp.loc[(temp.success==1)].time



train_test_df=train_test_df.join(temp.set_index('index')['last_informational_completed'],on='index')


train_test_df.last_informational_completed=train_test_df.groupby('person',
                                            sort=False).last_informational_completed.shift(periods=1)
train_test_df.last_informational_completed=train_test_df.groupby('person',
                                            sort=False).last_informational_completed.fillna(method='ffill')

In [127]:
train_test_df.fillna(-1,inplace=True)

In [128]:
#save the dataframe
train_test_df.to_csv('data.csv',index=False)
